# Prepare the FREEBIORYS4V2 data for the CREG025.L75 configuration BDYs 

In [98]:
import xarray as xr
import pandas as pd
import numpy as np
from netCDF4 import Dataset
import matplotlib.pylab as plt
import matplotlib as mpl
from matplotlib import rcParams
from mpl_toolkits.basemap import Basemap
from fsspec.implementations.local import LocalFileSystem
fs = LocalFileSystem()
import os 

%matplotlib inline

In [99]:
from kerchunk.hdf import SingleHdf5ToZarr 
from kerchunk.combine import MultiZarrToZarr

import fsspec
import json
import intake

import dask
import dask_hpcconfig
from distributed import Client
from dask.distributed import Client, LocalCluster


ModuleNotFoundError: No module named 'kerchunk'

In [3]:
cluster = dask_hpcconfig.cluster("datarmor", **{})
client = Client(cluster)
cluster.scale(jobs=3)

/home1/datawork/ctalandi/conda-env/Jupyter-Dask/lib/python3.11/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)


In [4]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.0.106:8787/status,
Dashboard: http://10.148.0.106:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.106:55808,Workers: 0
Dashboard: http://10.148.0.106:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


>> Function definition START

In [ ]:
def Arc_Bat(ztype='isol1000') :

        locpath='/home/datawork-lops-drakkarcom/SIMULATION-OUTPUTS/FREDY/CONFIGS/CREG12.L75/GRID/'
        locfile='bathymetry_CREG12_V3.3_CT20180612Larger_ct20190102.nc'
        fieldbat=Dataset(locpath+locfile)
        lon  = np.squeeze(fieldbat.variables['nav_lon'])
        lat  = np.squeeze(fieldbat.variables['nav_lat'])
        My_var = np.squeeze(fieldbat.variables['Bathymetry'])

        spval = 0.
        My_var= np.ma.masked_where(My_var <= spval,My_var)

        if ztype == 'isol1000' :
                vmin=1000. ; vmax=2000.
                contours=[1000.]
                limits=[vmin,vmax]
                myticks=[1000.]
        elif ztype == 'isol1500' :
                vmin=1500. ; vmax=2000.
                contours=[1500.]
                limits=[vmin,vmax]
                myticks=[1500.]
        elif ztype == 'isomonarc' :
                vmin=500. ; vmax=4000.
                contours=[500.,2000.,4000.]
                limits=[vmin,vmax]
                myticks=[500.,2000.,4000.]
        elif ztype == 'isol500' :
                vmin=500. ; vmax=500.
                contours=[500.]
                limits=[vmin,vmax]
                myticks=[500.]
        else:
                vmin=0. ; vmax=8000.
                contours=[100.,500.,1000.,2000.,3000.,3500.,4000.]
                limits=[vmin,vmax]
                myticks=[100.,500.,1000.,2000.,3000.,3500.,4000.]

        #
        #rcParams['text.latex.unicode']=True
        plt.rcParams['contour.negative_linestyle'] = 'solid'
        #
        m = Basemap(projection='npstere',boundinglat=65,lon_0=-60, resolution='i')
        norm = mpl.colors.Normalize(vmin=limits[0], vmax=limits[1])
        pal = plt.cm.get_cmap('binary')
        X,Y = m(lon,lat)

        # contour (optional)
        CS2 = m.contour(X, Y, My_var, linewidths=0.5,levels=contours, colors='black', alpha=1.)
        plt.clabel(CS2, CS2.levels, inline=True, fmt='%.0f', fontsize=3)

        return m, X, Y


In [3]:
def Arc_plot(lon,lat,tab,contours,limits,myticks=None,name=None,zmy_cblab=None,zmy_cmap=None,filename='test.pdf',zvar=None):
        #
# Les 2 lignes suivantes posent un probleme a l'execution lie a LaTex ....
#       rcParams['text.usetex']=True
#       rcParams['text.latex.unicode']=True
        #rcParams['font.family']='serif'

        plt.rcParams['contour.negative_linestyle'] = 'solid'
        #
        m = Basemap(projection='npstere',boundinglat=65,lon_0=-60, resolution='i')

        if zvar == 'sivolu' or zvar == 'siconc'  or zvar == 'ssh' or zvar == 'socurl' :
                zfontsize=10.
        else:
                zfontsize=6.

        if zvar != 'Bathy' :
                m.drawparallels(np.arange(-90.,91.,5.),labels=[False,False,False,False], size=zfontsize, linewidth=0.3)
                m.drawmeridians(np.arange(-180.,181.,20.),labels=[True,False,False,True], size=zfontsize, latmax=90.,linewidth=0.3)
                m.fillcontinents(color='grey',lake_color='white')

        norm = mpl.colors.Normalize(vmin=limits[0], vmax=limits[1])

        if zmy_cmap != None :
                pal = zmy_cmap
        else:
                pal = plt.cm.get_cmap('coolwarm')
                #pal = plt.cm.get_cmap('terrain')

        X,Y = m(lon,lat)
        C = m.contourf(X,Y,tab,contours,cmap=pal,norm=norm,extend='both')
        if zvar == 'ssh':
                CS=m.contour(X, Y, tab, linewidths=0.5, levels=npy.arange(limits[0],limits[1],5.), colors='k', alpha=0.4)

        ############################################################################################################
        ############################################################################################################
        moorplot=1
        if moorplot == 1 :
                bx_ARCB={'name':'B'  ,'lon_min':-150.,'lon_max':-150.,'lat_min':78.,'lat_max':78.}
                bx_ARCM={'name':'M1' ,'lon_min': 125.,'lon_max': 125.,'lat_min':78.,'lat_max':78.}
                bx_EURA={'name':'EUR','lon_min':  60.,'lon_max':  60.,'lat_min':85.,'lat_max':85.}

                All_box=[bx_ARCB,bx_EURA]
                for box in All_box:
                        lats = [box['lat_min'],box['lat_max']]
                        lons = [box['lon_min'],box['lon_max']]
                        x,y = m(lons,lats)
                        m.scatter(x,y,3,marker='o', color='r')
                        #m.plot(x,y,linewidth=2, color='g')
        ############################################################################################################
        ############################################################################################################

        # colorbar      
        if myticks is None:
                cbar = plt.colorbar(C,format='%.2f',orientation='vertical',shrink=0.8)
        else:
                if zvar == 'votemper' or zvar == 'vosaline' or zvar == 'sivolu' :
                        cbar = plt.colorbar(C,format='%.2f',orientation='vertical',shrink=0.8,drawedges=True)
                else:
                        cbar = plt.colorbar(C,format='%.0f',orientation='vertical',shrink=0.8,drawedges=True)

                cbar.set_label(zmy_cblab,fontsize=zfontsize)
                cl = plt.getp(cbar.ax, 'ymajorticklabels')
                plt.setp(cl, fontsize=zfontsize)

        plt.title(name,fontsize=zfontsize)

        return

In [4]:
def time_axis(in_array,lyear):    # Rebuild the time axis 
    new_time=pd.date_range(start=str(lyear)+'-01-01',periods=in_array.dims['time_counter'],freq='H',inclusive='left')
    #print('        Year treated '+str(year)+' has '+str(ds_skt.dims['time_counter'])+' hours')
    return new_time

>> Function definition END

# CREG025.L75  Grid

In [5]:
GRIDPATH='/home1/scratch/ctalandi/CREG025.L75/CREG025.L75-I/'
ds_grd=xr.open_dataset(GRIDPATH+'CREG025.L75_byte_mask.nc')
ds_grd
tmask2D=ds_grd['tmask'].isel(z=0,t=0)

FileNotFoundError: [Errno 2] No such file or directory: b'/home1/scratch/ctalandi/CREG025.L75/CREG025.L75-I/CREG025.L75_byte_mask.nc'

# CREG025.L75  Bathymetry

In [6]:
locpath='/home/datawork-lops-drakkarcom/SIMULATION-OUTPUTS/FREDY/CONFIGS/CREG025.L75/GRID/'
locfile='CREG025.L75_domain_cfg.nc'
fieldbat=xr.open_dataset(locpath+locfile)

FileNotFoundError: [Errno 2] No such file or directory: b'/home/datawork-lops-drakkarcom/SIMULATION-OUTPUTS/FREDY/CONFIGS/CREG025.L75/GRID/CREG025.L75_domain_cfg.nc'

In [90]:
CONFIG='FREEBIORYS2V4'
bgcvar_lst=['alk', 'bfe', 'caco3', 'chl', 'dchl', 'dfe', 'dic', 'doc', 'dsi', 'fe', 'goc', 'gsi', \
           'nchl', 'nfe', 'nh4', 'no3', 'nppv', 'o2', 'par', 'phy2', 'phyc', 'phy', 'po4', 'poc', \
           'sfe', 'si', 'zoo2', 'zoo']

# Read the original data downloaded

In [103]:
my_year=1994

In [104]:
# Define all areas
area_bering={'AREA':'BERING'     , 'imin' :3 , 'imax':25 ,'jmin':8 , 'jmax':15}
area_subtro={'AREA':'SUBTROPGYRE', 'imin' :17, 'imax':288,'jmin':11, 'jmax':21}

#lst_dom=[area_bering,area_subtro]
lst_dom=[area_bering,area_subtro]

In [105]:
PATHIN='/data0/project/drakkar/CONFIGS/ORCA025.L75/FREEBIORYS4V2/INIT/'
OUTDIR='/data0/project/drakkar/CONFIGS/ORCA025.L75/FREEBIORYS4V2/MODIFIED/'

for dom_extra in lst_dom:
  
    print()
    print(' The Sub-area considered is:', dom_extra['AREA'])

    TARDIR=PATHIN+dom_extra['AREA']+'/'+str(my_year)
    
    # Loop on variables 
    for var in bgcvar_lst:

        print(' >>>>>>>>>       The variable considered is:', var)
        
        # Read data base
        infile=CONFIG+'_'+dom_extra['AREA']+'_1m_y'+str(my_year)+'_'+var+'.nc'
        ds_full=xr.open_dataset(TARDIR+'/'+infile)
        # Exrtact the specific area
        imin=dom_extra['imin'] ; imax=dom_extra['imax']
        jmin=dom_extra['jmin'] ; jmax=dom_extra['jmax']
        fit_data=ds_full.isel({'x':slice(imin,imax),'y':slice(jmin,jmax)})
        if dom_extra['AREA'] == 'BERING':
            # Apply specific change along Bering to fit NEMO BDYs requirements 
            fit_data=fit_data.isel(x=slice(None, None, -1))
        # Save new dataset 
        oufile=CONFIG+'_'+dom_extra['AREA']+'_1m_y'+str(my_year)+'_'+var+'.nc'
        fit_data.to_netcdf(OUTDIR+dom_extra['AREA']+'/'+str(my_year)+'/'+oufile, unlimited_dims='time_counter')
             


 The Sub-area considered is: BERING
 >>>>>>>>>       The variable considered is: alk
 >>>>>>>>>       The variable considered is: bfe
 >>>>>>>>>       The variable considered is: caco3
 >>>>>>>>>       The variable considered is: chl
 >>>>>>>>>       The variable considered is: dchl
 >>>>>>>>>       The variable considered is: dfe
 >>>>>>>>>       The variable considered is: dic
 >>>>>>>>>       The variable considered is: doc
 >>>>>>>>>       The variable considered is: dsi
 >>>>>>>>>       The variable considered is: fe
 >>>>>>>>>       The variable considered is: goc
 >>>>>>>>>       The variable considered is: gsi
 >>>>>>>>>       The variable considered is: nchl
 >>>>>>>>>       The variable considered is: nfe
 >>>>>>>>>       The variable considered is: nh4
 >>>>>>>>>       The variable considered is: no3
 >>>>>>>>>       The variable considered is: nppv
 >>>>>>>>>       The variable considered is: o2
 >>>>>>>>>       The variable considered is: par
 >>>>>>>>>       The variable

In [16]:
client.close()
cluster.close()

/home1/datawork/ctalandi/conda-env/Jupyter-Dask/lib/python3.11/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
